In [14]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

# Reading input train and test data
train = pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

# Fill NA values
train=train.fillna(value=-10)
test=test.fillna(value=-10)

# get response variable
target=train['Active_Customer']
train.drop(['Active_Customer','Cust_id'],axis=1,inplace=True)

# saving test id
testID=test['Cust_id']
test.drop(['Cust_id'],axis=1,inplace=True)

# Checking test and train descriptions
#train.describe()
#test.describe()

# collecting the number of train instances
num_train=train.shape[0]

# concatenate train and test to a single object
df=pd.concat([train,test])

#df.describe()

# Converting dtype of character objects to categorical values
for column in df:
    if str(df[column].dtype)=='object':
        print(column,str(df[column].dtype))
        df[column] = df[column].astype('category')       

('Cust_status', 'object')
('Trans24', 'object')
('Trans25', 'object')
('Trans26', 'object')
('Trans27', 'object')


In [15]:
# Dummy encoding for the categorical variables
# drop_first not set for 0.17 version of pandas
df=pd.get_dummies(df,drop_first=True)
df.describe()

# Fill NA values
df=df.fillna(value=-10)


# Collecting the promotion columns
promotion_index=[]
id=0
for column in df:
    if column.find('Promotion')!=-1:
        promotion_index.append(id)
    id=id+1
print id
print promotion_index

255
[202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249]


In [16]:
# Functions,code for feature engineering

def getCounts(x):
    count=0
    for i in x:
        if i==0:
            count=count+1
    return count

def naCounts(x):
    count=0
    for i in x:
        if i==-10:
            count=count+1
    return count

def getnum_promotions(x,promotion_index):
    count=0
    for i in promotion_index:
        if x[i]!=-10 and x[i]>0:
            count=count+1
    return count

def getsum_promotions(x,promotion_index):
    sum=0
    for i in promotion_index:
        if x[i]!=-10 and x[i]>0:
            sum=sum+x[i]
    return sum



#sum of zeros across the instances
df['zeroCounts']=df.apply(getCounts,axis=1)



#sum of NAs across the instances
df['negativeCounts']=df.apply(naCounts,axis=1)


#sum of promotions across the instances
df['promotionCounts']=df.apply(lambda x : getnum_promotions(x,promotion_index),axis=1)


#totalnumber of promotions not equal to zeros
df['sum_promotionCounts'] =df.apply(lambda x : getsum_promotions(x,promotion_index),axis=1)

interactions2way=[('col1','col2')]

for A, B in interactions2way:
    feat = "_".join([A, B])
    df[feat] = df[A] - df[B]




In [3]:
# # Block to check K Nearest Neighbour Classifier
# from sklearn.preprocessing import MinMaxScaler
# from sklearn import neighbors
# df = MinMaxScaler().fit_transform(df)
#clf = neighbors.KNeighborsClassifier(n_neighbors=1000, weights='distance',algorithm='auto',n_jobs=2)

gbt= GradientBoostingClassifier(loss='deviance',learning_rate=0.02, subsample=0.9, 
                               random_state=1,max_depth=6, n_estimators=160,verbose=1)
scores=cross_validation.cross_val_score(gbt,df.iloc[0:num_train,],target,cv=5,scoring='accuracy')
print scores
print("Accuracy: %f (+/- %f)" % (scores.mean(), scores.std() * 2))

In [17]:
# Training a extra trees classifier

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn import cross_validation

et1 = ExtraTreesClassifier(n_estimators=600,max_features= 80,criterion= 'entropy',min_samples_split= 2,
                          max_depth= 60, min_samples_leaf= 2,random_state=1,
                          verbose=1,n_jobs = 3)

et2= ExtraTreesClassifier(n_estimators=600,max_features= 80,criterion= 'gini',min_samples_split= 2,
                          max_depth= 60, min_samples_leaf= 2,random_state=1,
                          verbose=1,n_jobs = 3)

gbt1= GradientBoostingClassifier(loss='deviance',learning_rate=0.02, subsample=0.9, 
                               random_state=1,max_depth=6, n_estimators=160,verbose=1)

gbt2= GradientBoostingClassifier(loss='deviance',learning_rate=0.01, subsample=0.8, 
                               random_state=2,max_depth=5, n_estimators=100,verbose=1)

rf1= RandomForestClassifier(n_estimators=600, n_jobs=3,max_features=40,max_depth=60,criterion='entropy',
                           random_state=1,verbose=1)

rf2= RandomForestClassifier(n_estimators=600, n_jobs=3,max_features=20,criterion='gini',
                           random_state=5,verbose=1)

eclf = VotingClassifier(estimators=[('c1', et1), ('c2', gbt1),
                                    ('c3', rf1),('b1', et1), ('b2', gbt2),
                                    ('b3', rf2)],
                        voting='soft', weights=[0.1,0.3,0.1,0.1,0.3,0.1])

import numpy as np
predictions=np.zeros((num_train))
print predictions.shape[0]

25766


In [18]:
# Selecting startified sampling
nfolds=5
skf = list(StratifiedKFold(target,nfolds,random_state=2))

for i, (trainIndex, testIndex) in enumerate(skf):
    print "Fold", i+1
    X_train = df.iloc[trainIndex,]
    y_train = target[trainIndex]
    X_test = df.iloc[testIndex,]
    y_test = target[testIndex]
    eclf.fit(X_train, y_train)
    predictions[testIndex] = eclf.predict_proba(X_test)[:,1]
    
for i in range(0,predictions.shape[0]):
    if predictions[i]>0.5:
        predictions[i]=1
    else:
        predictions[i]=0  
print metrics.accuracy_score(predictions,target)

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    8.6s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   39.6s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  1.6min
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:  2.1min finished


Fold 1
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3784           0.0072            2.32m
         2           1.3707           0.0065            2.25m
         3           1.3635           0.0059            2.21m
         4           1.3562           0.0056            2.25m
         5           1.3497           0.0056            2.25m
         6           1.3433           0.0058            2.24m
         7           1.3369           0.0053            2.21m
         8           1.3308           0.0058            2.19m
         9           1.3248           0.0051            2.16m
        10           1.3190           0.0049            2.14m
        20           1.2709           0.0039            1.98m
        30           1.2337           0.0024            1.85m
        40           1.2038           0.0018            1.72m
        50           1.1802           0.0010            1.57m
        60           1.1638           0.0008            1.42m


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    6.7s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   29.8s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  1.2min
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:  1.6min finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    9.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   39.2s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  1.5min
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:  2.0min finished



      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3827           0.0034           53.46s
         2           1.3791           0.0032           53.12s
         3           1.3758           0.0033           57.23s
         4           1.3723           0.0030           59.02s
         5           1.3691           0.0031           58.90s
         6           1.3659           0.0030           58.55s
         7           1.3623           0.0029           57.65s
         8           1.3595           0.0028           56.50s
         9           1.3564           0.0028           55.75s
        10           1.3534           0.0027           54.55s
        20           1.3261           0.0024           46.16s
        30           1.3009           0.0019           39.32s
        40           1.2801           0.0015           33.69s
        50           1.2623           0.0013           28.11s
        60           1.2495           0.0012           22.33s
      

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    2.8s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   13.0s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:   29.7s
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:   39.9s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:    0.4s
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:    0.6s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:    0.4s
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:    0.5s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:    0.4s
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:    0.

...

 5
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3785           0.0072            2.57m
         2           1.3710           0.0068            2.52m
         3           1.3635           0.0058            2.49m
         4           1.3567           0.0063            2.48m
         5           1.3501           0.0059            2.49m
         6           1.3434           0.0056            2.45m
         7           1.3373           0.0053            2.40m
         8           1.3313           0.0054            2.40m
         9           1.3248           0.0042            2.46m
        10           1.3192           0.0046            2.43m
        20           1.2713           0.0032            2.26m
        30           1.2335           0.0020            2.02m
        40           1.2067           0.0014            1.83m
        50           1.1827           0.0009            1.66m
        60           1.1637           0.0006            1.50m
    

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    7.6s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   33.9s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  1.3min
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:  1.7min finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   10.2s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   51.2s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  1.9min
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:  2.5min finished



      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3826           0.0032           59.30s
         2           1.3791           0.0031           58.70s
         3           1.3758           0.0032           57.68s
         4           1.3724           0.0031           58.30s
         5           1.3690           0.0029           57.95s
         6           1.3659           0.0030           57.07s
         7           1.3627           0.0030           56.37s
         8           1.3595           0.0028           55.61s
         9           1.3567           0.0028           54.93s
        10           1.3534           0.0028           54.23s
        20           1.3261           0.0023           47.93s
        30           1.3019           0.0018           42.11s
        40           1.2820           0.0016           36.43s
        50           1.2660           0.0013           33.28s
        60           1.2492           0.0010           27.16s
      

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    3.1s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   13.7s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:   31.1s
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:   42.2s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.2s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:    0.5s
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:    0.7s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:    0.4s
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:    0.6s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:    0.5s
[Parallel(n_jobs=3)]: Done 600 out of 600 | elapsed:    0.


0.682022820772
